# Titanic

El **RMS Titanic** fue un transatlántico británico, el mayor barco de pasajeros del mundo al finalizar su construcción, que se hundió durante la noche del 14 y la madrugada del 15 de abril de 1912 durante su viaje inaugural desde Southampton a Nueva York. 

En el hundimiento del Titanic murieron 1496 personas de las 2208 que iban a bordo, lo que convierte a esta catástrofe en uno de los mayores naufragios de la historia ocurridos en tiempos de paz.

El naufragio del Titanic conmocionó e indignó al mundo entero por el elevado número de víctimas mortales y por los errores cometidos en el accidente. Las investigaciones públicas realizadas en Reino Unido y los Estados Unidos llevaron a la implementación de importantes mejoras en la seguridad marítima y a la creación en 1914 del _Convenio Internacional para la Seguridad de la Vida Humana en el Mar_ (SOLAS, por sus siglas en inglés), que todavía hoy rige la seguridad marítima.

Se ha podido documentar algo de información relacionado con la tragedia. La enciclopedia británica detalla información de 1,310 pasajeros, lo que permite hacer un poco de trabajo estadístico.

Los campos disponibles son los siguientes:

- _clase_viaje_: Es la clase socio-económica en la que viajaba el pasajero.
- _sobrevivencia_: Indica si el pasajero sobrevivió o no. (0 = Murió; 1 = Vivió)
- _nombre_: Indica el nombre registrado del pasajero.
- _sexo_: Indica el sexo del pasajero.
- _edad_: Indica la edad en años del pasajero. En caso de que la edad sea menor a 1, indica que era menor a esa edad.
- _pariente_: Indica cuantos parientes políticos o esposa, viajaban con el pasajero.
- _familiar_: Indica cuántos parientes cosanguíneos (padres, hijos) viajaban con el pasajero.
- _boleto_: Número de ticket con el que viajaba el pasajero.
- _tarifa_: Costo (en libras esterlinas), del boleto pagado.
- _cabina_: Cabina o cabinas ocupadas por el pasajero.
- _embarque_: Clave del puerto de embarcación del pasajero. (C = Cherbourg; Q = Queenstown; S = Southampton)
- _cuerpo_: Número de identificación de cuerpo. Solo en caso de haber muerto.
- _residencia_: Hogar/Destino.

# Objetivo del análisis

Se desea hacer un trabajo de analítica que muestre qué tanto influyó la clase en la que viajaba, el sexo, la edad y si viajaban solos o acompañados, en las posibilidades de sobrevivencia de los pasajeros.

Supondríamos que:
1. "Mujeres y niños primero", entonces las mujeres y los niños se salvaron más que los hombres adultos.
2. Los de primera clase, se salvaron más, porque tuvieron acceso preferencial a botes salvaidas.
3. La gente que viajó acompañada, al tener que preocuparse por otros, se salvó menos.

## Cargando librerías y datos

In [1]:
# Se importan las librerías básicas para el trabajo de analítica de datos

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# Para que nuestros gráficos se integren a Jupyter Notebook
%matplotlib inline



In [2]:
# Cargar los datos de un archivo, y ponerlos en una variable de tipo dataframe de pandas

df = pd.read_csv('titanic_sp.csv')

# La variable df representa al conjunto de datos completo.

In [3]:
# Declarando una función para dibujar una línea separadora

def linea():
    print(' ')
    print('--------------------------------------------------')
    print(' ')

linea()


 
--------------------------------------------------
 


## Volúmenes y tipos de datos

In [4]:
# ¿Cuántos registros tiene nuestra fuente de datos?

print(len(df))

1310


In [5]:
# ¿Cuántas columnas tengo?

print(len(df.columns))

14


In [6]:
# En un solo paso

df.shape

(1310, 14)

In [7]:
# Qué columnas tengo y qué tipo de dato tienen

df.dtypes

clase_viaje        int64
sobrevivencia      int64
nombre            object
sexo              object
edad             float64
parientes          int64
familiares         int64
boleto            object
tarifa           float64
cabina            object
embarque          object
bote              object
cuerpo           float64
residencias       object
dtype: object

In [8]:
# Dato ausente (missing): Dato requerido por los trabajos de análisis, 
# que no está disponible por alguna razón.

# Para identificarlos, hay que saber qué queremos analizar, y qué requerimos.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clase_viaje    1310 non-null   int64  
 1   sobrevivencia  1310 non-null   int64  
 2   nombre         1310 non-null   object 
 3   sexo           1310 non-null   object 
 4   edad           1047 non-null   float64
 5   parientes      1310 non-null   int64  
 6   familiares     1310 non-null   int64  
 7   boleto         1310 non-null   object 
 8   tarifa         1309 non-null   float64
 9   cabina         295 non-null    object 
 10  embarque       1308 non-null   object 
 11  bote           486 non-null    object 
 12  cuerpo         121 non-null    float64
 13  residencias    745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.4+ KB


In [9]:
linea()
print("Los campos disponibles son:")
for campo_origen in df.columns.tolist():
    print(campo_origen)

linea()
print("Los campos requeridos son:")
campos_requeridos=["clase_viaje", "sobrevivencia", "sexo", "edad", "parientes", "familiares"]
for campo in campos_requeridos:
    print(campo)

 
--------------------------------------------------
 
Los campos disponibles son:
clase_viaje
sobrevivencia
nombre
sexo
edad
parientes
familiares
boleto
tarifa
cabina
embarque
bote
cuerpo
residencias
 
--------------------------------------------------
 
Los campos requeridos son:
clase_viaje
sobrevivencia
sexo
edad
parientes
familiares


In [10]:
# Eliminar campos innecesarios. Yo quiero a) leer secuencialmente todos los campos de mi dataframe (df)
# y b) en caso de que no estén dentro de los campos requeridos (campos_requeridos), entonces, c) borrar el campo (del).

for campo in df.columns.tolist(): # a)
    if campo not in campos_requeridos: # b)
        del df[campo] # c)

# Revisar cómo quedó
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clase_viaje    1310 non-null   int64  
 1   sobrevivencia  1310 non-null   int64  
 2   sexo           1310 non-null   object 
 3   edad           1047 non-null   float64
 4   parientes      1310 non-null   int64  
 5   familiares     1310 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 61.5+ KB


In [11]:
# Eliminar observaciones que presentan datos ausentes. En este caso
# se genera un nuevo dataframe llamado <<pasajeros>>, que contiene las
# observaciones del data frame df, siempre y cuando edad no sea nula.
print(df[df["edad"].notnull()])
pasajeros=df[df["edad"].notnull()]

      clase_viaje  sobrevivencia    sexo     edad  parientes  familiares
0               1              1   mujer  29.0000          0           0
1               1              1  hombre   0.9167          1           2
2               1              0   mujer   2.0000          1           2
3               1              0  hombre  30.0000          1           2
4               1              0   mujer  25.0000          1           2
...           ...            ...     ...      ...        ...         ...
1304            3              0   mujer  14.5000          1           0
1306            3              0  hombre  26.5000          0           0
1307            3              0  hombre  27.0000          0           0
1308            3              0  hombre  29.0000          0           0
1309            3              0  hombre  29.0000          0           0

[1047 rows x 6 columns]


In [12]:
pasajeros.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1047 entries, 0 to 1309
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clase_viaje    1047 non-null   int64  
 1   sobrevivencia  1047 non-null   int64  
 2   sexo           1047 non-null   object 
 3   edad           1047 non-null   float64
 4   parientes      1047 non-null   int64  
 5   familiares     1047 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 57.3+ KB


Una vez que se eliminan las observaciones con datos ausentes, sigue eliminar observaciones con datos atípicos (si lo requiere la investigación). Para saber cuáles son atípicos, se sigue esta regla:

1. Cualquier dato por abajo del **mínimo no atípico** o arriba del **máximo no atípico** se considera atípico.
2. El **mínimo no atípico** es el resultado de restarle a la media 1.5 veces la desviación estándar.
3. El **máximo no atípico** es el resultado de sumarle a la media 1.5 veces la desviación estándar.

(Regla de Tukey)


In [13]:
# Determinación de rango no atípico

promedio_edad = pasajeros["edad"].mean()
desviacion_estandar_edad = pasajeros["edad"].std()
minimo_no_atipico = promedio_edad-(1.5*desviacion_estandar_edad)
maximo_no_atipico = promedio_edad+(1.5*desviacion_estandar_edad)

print("Mínimo no atípico:", minimo_no_atipico)
print("Máximo no atípico:", maximo_no_atipico)



Mínimo no atípico: 8.270341976454851
Máximo no atípico: 51.490243887919554


In [14]:
# Eliminar observaciones con valores atípicos

pasajeros=pasajeros[(pasajeros["edad"]>minimo_no_atipico) & (pasajeros["edad"]<maximo_no_atipico)]

In [15]:
pasajeros.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 888 entries, 0 to 1309
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clase_viaje    888 non-null    int64  
 1   sobrevivencia  888 non-null    int64  
 2   sexo           888 non-null    object 
 3   edad           888 non-null    float64
 4   parientes      888 non-null    int64  
 5   familiares     888 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 48.6+ KB


In [16]:
# Ver información

pasajeros.head()

,clase_viaje,sobrevivencia,sexo,edad,parientes,familiares
0,1,1,mujer,29.0,0,0
3,1,0,hombre,30.0,1,2
4,1,0,mujer,25.0,1,2
5,1,1,hombre,48.0,0,0
7,1,0,hombre,39.0,0,0


In [17]:
pasajeros.tail()

,clase_viaje,sobrevivencia,sexo,edad,parientes,familiares
1304,3,0,mujer,14.5,1,0
1306,3,0,hombre,26.5,0,0
1307,3,0,hombre,27.0,0,0
1308,3,0,hombre,29.0,0,0
1309,3,0,hombre,29.0,0,0


In [18]:
# Se eliminan también registros duplicados (si aplica)

# Eliminar datos duplicados (mismos valores en todos los campos)
# inplace=True provoca que el dataframe resultante (temporal), sustituya al dataframe de origen.
# lo que representa que la operación se realiza sobre el mismo dataframe.

pasajeros_sin_repetidos=pasajeros.drop_duplicates()

pasajeros_sin_repetidos.info()

# Dado el análisis que queremos hacer, eliminar repetidos ensucia los datos.
# No lo usaremos.

# ¿Está bien o no está bien que haya duplicados? ¿Es normal que haya duplicados, o no?

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539 entries, 0 to 1306
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clase_viaje    539 non-null    int64  
 1   sobrevivencia  539 non-null    int64  
 2   sexo           539 non-null    object 
 3   edad           539 non-null    float64
 4   parientes      539 non-null    int64  
 5   familiares     539 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 29.5+ KB


## Manejo de categoricos y campos calculados

1. Si se  tienen campos categóricos numericos clase_viaje y sobrevivencia
2. Quiero un campo categorico, llamado compañia, que indicque si la persona viajaba


In [40]:
# Paso 1 :crear dataframe que contiene categorico
cat_clase_viaje=pd.DataFrame({
    
    "clase_viaje":[1,2,3],
    "clase":["primera CLASE","SEGUNDA CLASE","TERCERA CLASE"]
})

# PASO 2 HACER EL MERGE CON EL DATAFFRAME BASE

pasajeros_completos = pasajeros.merge(cat_clase_viaje,on = "clase_viaje",how="inner")

#Paso 3 Comprobar que ya tenemos la tabla la union de los campos de los dos
#Dataframe
pasajeros_completos.head(10)


,clase_viaje,sobrevivencia,sexo,edad,parientes,familiares,clase
0,1,1,mujer,29.0,0,0,primera CLASE
1,1,0,hombre,30.0,1,2,primera CLASE
2,1,0,mujer,25.0,1,2,primera CLASE
3,1,1,hombre,48.0,0,0,primera CLASE
4,1,0,hombre,39.0,0,0,primera CLASE
5,1,0,hombre,47.0,1,0,primera CLASE
6,1,1,mujer,18.0,1,0,primera CLASE
7,1,1,mujer,24.0,0,0,primera CLASE
8,1,1,mujer,26.0,0,0,primera CLASE
9,1,0,hombre,24.0,0,1,primera CLASE


In [35]:
pasajeros_completos["acompañantes"] = pasajeros_completos["parientes"]+ pasajeros_completos["familiares"]

In [21]:
pasajeros_completos.head(10)


,clase_viaje,sobrevivencia,sexo,edad,parientes,familiares,clase,acompañantes
0,1,1,mujer,29.0,0,0,primera CLASE,0
1,1,0,hombre,30.0,1,2,primera CLASE,3
2,1,0,mujer,25.0,1,2,primera CLASE,3
3,1,1,hombre,48.0,0,0,primera CLASE,0
4,1,0,hombre,39.0,0,0,primera CLASE,0
5,1,0,hombre,47.0,1,0,primera CLASE,1
6,1,1,mujer,18.0,1,0,primera CLASE,1
7,1,1,mujer,24.0,0,0,primera CLASE,0
8,1,1,mujer,26.0,0,0,primera CLASE,0
9,1,0,hombre,24.0,0,1,primera CLASE,1


In [36]:
pasajeros_completos["compañia"] = np.where(pasajeros_completos["acompañantes"]==0,"sola","acompañanetes")
pasajeros_completos.head(10)


,clase_viaje,sobrevivencia,sexo,edad,parientes,familiares,clase,acompañantes,compañia
0,1,1,mujer,29.0,0,0,primera CLASE,0,sola
1,1,0,hombre,30.0,1,2,primera CLASE,3,acompañanetes
2,1,0,mujer,25.0,1,2,primera CLASE,3,acompañanetes
3,1,1,hombre,48.0,0,0,primera CLASE,0,sola
4,1,0,hombre,39.0,0,0,primera CLASE,0,sola
5,1,0,hombre,47.0,1,0,primera CLASE,1,acompañanetes
6,1,1,mujer,18.0,1,0,primera CLASE,1,acompañanetes
7,1,1,mujer,24.0,0,0,primera CLASE,0,sola
8,1,1,mujer,26.0,0,0,primera CLASE,0,sola
9,1,0,hombre,24.0,0,1,primera CLASE,1,acompañanetes


In [23]:
pasajeros_completos.describe(
)

,clase_viaje,sobrevivencia,edad,parientes,familiares,acompañantes
count,888.000000,888.000000,888.000000,888.000000,888.000000,888.000000
mean,2.256757,0.389640,29.136261,0.424550,0.347973,0.772523
std,0.828916,0.487943,9.872982,0.820362,0.815214,1.344767
min,1.000000,0.000000,9.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,0.000000,0.000000,0.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,0.000000
75%,3.000000,1.000000,36.000000,1.000000,0.000000,1.000000
max,3.000000,1.000000,51.000000,8.000000,6.000000,10.000000


In [38]:
# LLama la tencion que clase_viaje y sobreviencia da apartamiento de numericos y realmente son categoricos
# Se tranforman a Object.
#para el cambio de dato se sigue estas sintaxis
# campo= campo.astype(nuevo_tipo)
pasajeros_completos["clase_viaje"] = pasajeros_completos["clase_viaje"].astype("object")
pasajeros_completos["sobrevivencia"] = pasajeros_completos["sobrevivencia"].astype("object")
pasajeros_completos.head(10)


,clase_viaje,sobrevivencia,sexo,edad,parientes,familiares,clase,acompañantes,compañia,l
0,1,1,mujer,29.0,0,0,primera CLASE,0,sola,mujerprimera CLASE
1,1,0,hombre,30.0,1,2,primera CLASE,3,acompañanetes,hombreprimera CLASE
2,1,0,mujer,25.0,1,2,primera CLASE,3,acompañanetes,mujerprimera CLASE
3,1,1,hombre,48.0,0,0,primera CLASE,0,sola,hombreprimera CLASE
4,1,0,hombre,39.0,0,0,primera CLASE,0,sola,hombreprimera CLASE
5,1,0,hombre,47.0,1,0,primera CLASE,1,acompañanetes,hombreprimera CLASE
6,1,1,mujer,18.0,1,0,primera CLASE,1,acompañanetes,mujerprimera CLASE
7,1,1,mujer,24.0,0,0,primera CLASE,0,sola,mujerprimera CLASE
8,1,1,mujer,26.0,0,0,primera CLASE,0,sola,mujerprimera CLASE
9,1,0,hombre,24.0,0,1,primera CLASE,1,acompañanetes,hombreprimera CLASE


In [37]:
pasajeros_completos["l"] = pasajeros_completos["sexo"]+ pasajeros_completos["clase"]
pasajeros_completos.head(10)


,clase_viaje,sobrevivencia,sexo,edad,parientes,familiares,clase,acompañantes,compañia,l
0,1,1,mujer,29.0,0,0,primera CLASE,0,sola,mujerprimera CLASE
1,1,0,hombre,30.0,1,2,primera CLASE,3,acompañanetes,hombreprimera CLASE
2,1,0,mujer,25.0,1,2,primera CLASE,3,acompañanetes,mujerprimera CLASE
3,1,1,hombre,48.0,0,0,primera CLASE,0,sola,hombreprimera CLASE
4,1,0,hombre,39.0,0,0,primera CLASE,0,sola,hombreprimera CLASE
5,1,0,hombre,47.0,1,0,primera CLASE,1,acompañanetes,hombreprimera CLASE
6,1,1,mujer,18.0,1,0,primera CLASE,1,acompañanetes,mujerprimera CLASE
7,1,1,mujer,24.0,0,0,primera CLASE,0,sola,mujerprimera CLASE
8,1,1,mujer,26.0,0,0,primera CLASE,0,sola,mujerprimera CLASE
9,1,0,hombre,24.0,0,1,primera CLASE,1,acompañanetes,hombreprimera CLASE


In [39]:
del pasajeros_completos["parientes"]
del pasajeros_completos["familiares"]